In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dt = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
dt

In [ ]:
import nltk
nltk.download()

In [ ]:
dt.head

In [ ]:
dt.shape

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [ ]:
sentences = dt['text']

In [ ]:
for x in dt['text'] :
     sentences = sent_tokenize(x)
     words = word_tokenize(x)


In [ ]:
from nltk.probability import FreqDist

fdist = FreqDist(words)
fdist.most_common(10)

In [ ]:
import matplotlib.pyplot as plt
fdist.plot(100)

In [ ]:
words_no_punc = []

for h in words:
    if h.isalpha():
        words_no_punc.append(h.lower())
        
print(words_no_punc)
print("\n")
print(len(words_no_punc))

In [ ]:
fdist = FreqDist(words_no_punc)
fdist.most_common(10)

In [ ]:
fdist.plot(1000)

In [ ]:
from nltk.corpus import stopwords

stopwords = stopwords.words("english")
print(stopwords)

In [ ]:
clean_words = []

for w in words_no_punc:
    if w not in stopwords:
        clean_words.append(w)
        
print(clean_words)
print("\n")
print(len(clean_words))

In [ ]:
fdist = FreqDist(clean_words)

fdist.most_common(10)

In [ ]:
fdist.plot(10)

In [ ]:
from wordcloud import WordCloud

wordcloud = WordCloud().generate(sentences[0])

plt.figure(figsize = (12,12))
plt.imshow(wordcloud)

plt.axis("off")
plt.show()

In [ ]:
grammar = "NP : {<DT>?<JJ>*<NN>}"

parser = nltk.RegexpParser(grammar)

output = parser.parse(tagged_words)
print(output)

output.draw()

In [ ]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

counter = Counter(df['target'].tolist())
top_10_varieties = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
df = df[df['text'].map(lambda x: x in top_10_varieties)]

description_list = df['text'].tolist()
varietal_list = [top_10_varieties[i] for i in df['target'].tolist()]
varietal_list = np.array(varietal_list)

count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(description_list)


tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

train_x, test_x, train_y, test_y = train_test_split(x_train_tfidf, varietal_list, test_size=0.3)

clf = MultinomialNB().fit(train_x, train_y)
y_score = clf.predict(test_x)

n_right = 0
for i in range(len(y_score)):
    if y_score[i] == test_y[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))